<a href="https://colab.research.google.com/github/pravin1907/Hackfinal/blob/main/Hackfinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio
import gradio as gr

def analyze_lab_report(pdf_file):
    # Step 1: Train the model
    model, scaler, feature_columns = train_disease_model()

    # Step 2: Save uploaded file temporarily
    pdf_path = pdf_file.name

    # Step 3: Extract lab values
    extracted_values = process_lab_report(pdf_path)

    if not extracted_values:
        return (
            "❌ Could not extract any lab values. Please upload a clear text or scanned PDF.",
            "",
            "",
        )

    # Step 4: Predict disease
    input_df = map_to_model_features(extracted_values, feature_columns)
    input_scaled = scaler.transform(input_df)
    prediction = model.predict(input_scaled)[0]
    probs = dict(zip(model.classes_, model.predict_proba(input_scaled)[0]))

    # Step 5: Prepare readable outputs
    extracted_text = "\n".join([f"{k}: {v}" for k, v in extracted_values.items()])
    prob_text = "\n".join([f"{k.capitalize()}: {v*100:.1f}%" for k, v in probs.items()])

    # Step 6: Health explanation
    explanations = []
    if "Hemoglobin" in extracted_values:
        hgb = extracted_values["Hemoglobin"]
        if hgb < 12:
            explanations.append("⚠️ Low Hemoglobin — Possible Anemia")
        elif hgb > 17:
            explanations.append("⚠️ High Hemoglobin — May indicate dehydration")
        else:
            explanations.append("✅ Hemoglobin is normal")

    if "glucose" in extracted_values:
        glucose = extracted_values["glucose"]
        if glucose > 140:
            explanations.append("⚠️ High Glucose — Diabetes risk detected")
        elif glucose < 70:
            explanations.append("⚠️ Low Glucose — Hypoglycemia risk")
        else:
            explanations.append("✅ Glucose level is normal")

    if "ap_hi" in extracted_values:
        bp_sys = extracted_values["ap_hi"]
        if bp_sys > 140:
            explanations.append("⚠️ High Blood Pressure — Cardiovascular risk")
        elif bp_sys < 90:
            explanations.append("⚠️ Low Blood Pressure — May cause dizziness")
        else:
            explanations.append("✅ Blood Pressure is normal")

    explanation_text = "\n".join(explanations) if explanations else "ℹ️ Not enough data for detailed analysis"

    # Step 7: Return all results
    return (
        f"🏥 Predicted Disease: {prediction.upper()}",
        f"📈 Confidence Scores:\n{prob_text}",
        f"📊 Extracted Values:\n{extracted_text}\n\n💡 Health Insights:\n{explanation_text}",
    )

# --- Gradio Interface ---
iface = gr.Interface(
    fn=analyze_lab_report,
    inputs=[gr.File(label="📤 Upload Lab Report (PDF)")],
    outputs=[
        gr.Textbox(label="Prediction", lines=2),
        gr.Textbox(label="Confidence", lines=6),
        gr.Textbox(label="Detailed Findings", lines=12),
    ],
    title="🩺 Integrated Lab Report → Disease Prediction System",
    description="Upload your lab report PDF — the system will automatically extract values and predict possible disease risk (Anemia, Cardio, Diabetes).",
    allow_flagging="never"
)

iface.launch(share=True)